In [1]:
import sys
sys.path.append("../")
from src.market.underlying.gbm_stock import BrownianStock
import torch

# Set random seed for reproducibility
torch.manual_seed(42)

# Create a BrownianStock instance
stock = BrownianStock(sigma=0.2, mu=0.05)

# Simulate one path for 20 trading days (about a month)
stock.simulate(n_paths=1, time_horizon=20/250)

# Print the generated spot price time series
print(stock.spot)

tensor([[1.0000, 1.0191, 1.0309, 1.0039, 1.0127, 1.0267, 1.0373, 1.0597, 1.0646,
         1.0556, 1.0491, 1.0525, 1.0379, 1.0386, 1.0354, 1.0468, 1.0429, 1.0378,
         1.0485, 1.0404, 1.0328]])


In [4]:
import numpy as np
import matplotlib.pyplot as plt

# Create a European option with our Brownian stock
from src.market.derivative.european_option import EuropeanOption
from src.nn.bs.european import BSEuropeanOption
from src.autogreek import delta, gamma, vega, theta

# Set random seed for reproducibility
torch.manual_seed(42)

# Create stock and option
stock = BrownianStock(sigma=0.2, mu=0.05)
option = EuropeanOption(
    underlier=stock,
    call=True,
    strike=1.0,
    maturity=20/250  # 20 trading days
)

# Create Black-Scholes pricer
bs = BSEuropeanOption(option)

# Simulate stock price
n_paths = 1
stock.simulate(n_paths=n_paths, time_horizon=20/250)

# Initialize hedging portfolio
initial_cash = 1.0  # Initial cash position
portfolio_value = torch.zeros_like(stock.spot)
cash_position = torch.zeros_like(stock.spot)
stock_position = torch.zeros_like(stock.spot)

# Set initial portfolio value
portfolio_value[0] = initial_cash

# Delta hedging simulation
for t in range(1, stock.spot.size(-1)):
    # Calculate current option price and delta
    current_spot = stock.spot[0, t]
    time_to_maturity = (stock.spot.size(-1) - t) * stock.dt
    
    # Get option price and delta using Black-Scholes
    k = option.strike                         # 1.0 in your set-up
    log_moneyness = torch.log(current_spot / k)
    option_price = bs.price(
        log_moneyness=log_moneyness,
        time_to_maturity=time_to_maturity,
        volatility=stock.sigma
    )
    
    option_delta = bs.delta(
        log_moneyness= log_moneyness,
        time_to_maturity=time_to_maturity,
        volatility=stock.sigma
    )
    
    # Update stock position (delta hedge)
    stock_position[0, t] = option_delta
    
    # Calculate cash position
    if t == 1:
        # Initial hedge
        cash_position[0, t] = initial_cash - option_delta * current_spot
    else:
        # Update cash position based on previous positions
        prev_spot = stock.spot[0, t-1]
        cash_position[0, t] = (
            cash_position[0, t-1] * (1 + 0.05 * stock.dt) -  # Interest on cash
            (option_delta - stock_position[0, t-1]) * current_spot  # Rebalancing cost
        )
    
    # Calculate portfolio value
    portfolio_value[0, t] = cash_position[0, t] + stock_position[0, t] * current_spot

# Calculate option payoff at maturity
payoff = option.payoff_fn()

# Plot results
plt.figure(figsize=(12, 8))

# Plot stock price
plt.subplot(2, 1, 1)
plt.plot(stock.spot[0].numpy(), label='Stock Price')
plt.title('Stock Price')
plt.legend()

# Plot portfolio value and option payoff
plt.subplot(2, 1, 2)
plt.plot(portfolio_value[0].numpy(), label='Portfolio Value')
plt.axhline(y=payoff[0].item(), color='r', linestyle='--', label='Option Payoff')
plt.title('Portfolio Value vs Option Payoff')
plt.legend()

plt.tight_layout()
plt.show()

# Print final results
print(f"Final Stock Price: {stock.spot[0, -1]:.4f}")
print(f"Option Payoff: {payoff[0]:.4f}")
print(f"Final Portfolio Value: {portfolio_value[0, -1]:.4f}")
print(f"Hedging Error: {portfolio_value[0, -1] - payoff[0]:.4f}")

ValueError: log_moneyness is required if derivative is not set at this initialization.